# 5 Modeling<a id='5_Modeling'></a>

## 5.1 Contents<a id='5.1_Contents'></a>
* [5 Modeling](#5_Modeling)
  * [5.1 Contents](#5.1_Contents)
  * [5.2 Introduction](#5.2_Introduction)
  * [5.3 Imports](#5.3_Imports)
  * [5.4 Load Model](#5.4_Load_Model)
  * [5.5 Load Data](#5.5_Load_Data)
  * [5.6 Refit Model On All Available Data (excluding Big Mountain)](#5.6_Refit_Model_On_All_Available_Data_(excluding_Big_Mountain))
  * [5.7 Calculate Expected Big Mountain Ticket Price From The Model](#5.7_Calculate_Expected_Big_Mountain_Ticket_Price_From_The_Model)
  * [5.8 Big Mountain Resort In Market Context](#5.8_Big_Mountain_Resort_In_Market_Context)
    * [5.8.1 Ticket price](#5.8.1_Ticket_price)
    * [5.8.2 Vertical drop](#5.8.2_Vertical_drop)
    * [5.8.3 Snow making area](#5.8.3_Snow_making_area)
    * [5.8.4 Total number of chairs](#5.8.4_Total_number_of_chairs)
    * [5.8.5 Fast quads](#5.8.5_Fast_quads)
    * [5.8.6 Runs](#5.8.6_Runs)
    * [5.8.7 Longest run](#5.8.7_Longest_run)
    * [5.8.8 Trams](#5.8.8_Trams)
    * [5.8.9 Skiable terrain area](#5.8.9_Skiable_terrain_area)
  * [5.9 Modeling scenarios](#5.9_Modeling_scenarios)
    * [5.9.1 Scenario 1](#5.9.1_Scenario_1)
    * [5.9.2 Scenario 2](#5.9.2_Scenario_2)
    * [5.9.3 Scenario 3](#5.9.3_Scenario_3)
    * [5.9.4 Scenario 4](#5.9.4_Scenario_4)
  * [5.10 Summary](#5.10_Summary)
  * [5.11 Further work](#5.11_Further_work)


## 5.2 Introduction<a id='5.2_Introduction'></a>

In this notebook, we now take our model for ski resort ticket price and leverage it to gain some insights into what price Big Mountain's facilities might actually support as well as explore the sensitivity of changes to various resort parameters. Note that this relies on the implicit assumption that all other resorts are largely setting prices based on how much people value certain facilities. Essentially this assumes prices are set by a free market.

We can now use our model to gain insight into what Big Mountain's ideal ticket price could/should be, and how that might change under various scenarios.

In [1]:
#okay cool so we're gonna see what Big Mountain in Montana can actually charge based on what others are doing and their
#parameters. We're gonna "explore the sensitivity of changes to arious resort parameters" - so that doesn't mean we're
#gonna like literally change the parameters and see what happens to the price, cuz obviously we don't know what would
#happen. I guess it just means explore the trends / relationships b/w certain params and price, maybe some do have a
#direct linear correlation, maybe others have no correlation, and maybe others have a non-linear correlation

#and it's saying the assumptions are free market conditions, i.e. that the price is being determined by supply &
#demand, i.e. by how much ppl value certain things (params)

#and always remember, we can never have ALL the data, we can only get a snapshot/representation of it
#and we can never know how ppl truly feel about something, even if there's data on it / they answer survey questions

## 5.3 Imports<a id='5.3_Imports'></a>

In [2]:
import pandas as pd
import numpy as np
import os
import pickle  #?
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import cross_validate

## 5.4 Load Model<a id='5.4_Load_Model'></a>

In [3]:
# This isn't exactly production-grade, but a quick check for development
# These checks can save some head-scratching in development when moving from
# one python environment to another, for example

expected_model_version = '1.0'  #what was this supposed to represent again?
model_path = '../models/ski_resort_pricing_model.pkl'
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    if model.version != expected_model_version:
        print("Expected model version doesn't match version loaded")
    if model.sklearn_version != sklearn_version:
        print("Warning: model created under different sklearn version")
else:
    print("Expected model not found")

Expected model version doesn't match version loaded


/Users/deens/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/deens/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/deens/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 1.0.1. This

In [4]:
# okay cool so these are just version checkers to warn someone in the beginning that something may be off / not be what
# they expect

## 5.5 Load Data<a id='5.5_Load_Data'></a>

In [5]:
ski_data = pd.read_csv('../data/ski_data_step3_features.csv')

In [6]:
#oh once again i gotta change it up to match what they're expecting:

ski_data = ski_data[ski_data.yearsOpen < 1000]

ski_data.loc[ski_data.Name=='Silverton Mountain', 'SkiableTerrain_ac'] = 1819  #crucial step that i'd forgotten!!! this single value alone was so extreme that it completely threw off the r2 score and i was scratching my head so hard wondering what the heck happened!!!! this came to light when i did the plot/histo of skiable acres and saw this crazy outlier and scrunched up graph way diff from the template instantly noticeable!

ski_data.dropna(subset=['AdultWeekend'], inplace=True)

ski_data.reset_index(drop=True, inplace=True)

In [7]:
big_mountain = ski_data[ski_data.Name == 'Big Mountain Resort']

In [8]:
big_mountain.T

,124
Name,Big Mountain Resort
Region,Montana
state,Montana
summit_elev,6817
vertical_drop,2353
base_elev,4464
trams,0
fastSixes,0
fastQuads,3
quad,2


In [9]:
#there we go, aH now iA should be good

## 5.6 Refit Model On All Available Data (excluding Big Mountain)<a id='5.6_Refit_Model_On_All_Available_Data_(excluding_Big_Mountain)'></a>

This next step requires some careful thought. We want to refit the model using all available data. But should we include Big Mountain data? On the one hand, we are _not_ trying to estimate model performance on a previously unseen data sample, so theoretically including Big Mountain data should be fine. One might first think that including Big Mountain in the model training would, if anything, improve model performance in predicting Big Mountain's ticket price. But here's where our business context comes in. The motivation for this entire project is based on the sense that Big Mountain needs to adjust its pricing. One way to phrase this problem: we want to train a model to predict Big Mountain's ticket price based on data from _all the other_ resorts! We don't want Big Mountain's current price to bias this. We want to calculate a price based only on its competitors.

In [10]:
#interesting, so when we want to predict/set one or a specific group of values, then the question is - do we include
#that/those in the TEST set, or do we leave that out and go to it independently/separately, after having worked on our
#model and testing it and being happy with it, and only then applying it to the target?

#cuz remember, the goal is to SET a fair price for Big Mountain.... ****NOT**** predict what it currently is!
#we don't care that much what it currently is -- the whole point is to set it RIGHT in light of what others in the
#industry are / what the free market is doing!

#aH that's exactly what they wrote too, which I came up w/ aH before reading it :)

In [11]:
#okay so our X's are all the features, except price, and excluding Big Mountain
#and our y's, the thing we're tryna predict, are the $prices ('AdultWeekend'), excluding of course Big Mountain's

#okay so here's what 'model' is:

# model_path = '../models/ski_resort_pricing_model.pkl'
# if the model_path exists, it's set to equal 'model'

# so:

# model = model_path = 'ski_resort_pricing_model.pkl' >> which is our set of dev/developer params!
# this is our 'best_model' which is 'rf_grid_cv.best_estimator_'
#which was: GridSearchCV(RF_pipe, param_grid=grid_params, cv=5, n_jobs=-1)

# so the X set/df is all the rows except Big Mountain, and all the columns for those except price...
# but wait a minute - HOW DOES IT KNOW NOT TO INCLUDE THE PRICE COLUMN?!
# like yes it's set to use the columns 'X_columns', but where does that come from and how does it know not to use
# the price column???

# oh, so remem in 'model', which we defined at the end of the last unit, we set the 'X_columns' param to equal the
# columns of X_train, which we THEN/THERE defined to be all the columns of ski_data EXCEPT AdultWeekend!!!

# and then the y set is of course just one column - the price$ - AdultWeekend, and for all the rows except Big Mountain!

X = ski_data.loc[ski_data.Name != "Big Mountain Resort", model.X_columns]
y = ski_data.loc[ski_data.Name != "Big Mountain Resort", 'AdultWeekend']

In [12]:
len(X), len(y)

(276, 276)

In [13]:
model.fit(X, y)

/Users/deens/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:388: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


ValueError: Invalid parameter min_impurity_split for estimator DecisionTreeRegressor(criterion='mse', max_features='auto'). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
#here we run the model multiple times to see how the negative MAE fares across multiple trials

#and again, 'model' is our latest, best, random forest model from the end of unit 4:
# rf_grid_cv.best_estimator_
#the best_estimator_ means store the combo of params that gave us the highest score, like in terms of prediction accuracy

cv_results = cross_validate(model, X, y, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

#this is like exactly what we did before, in end unit 4, no?:
# rf_neg_mae = cross_validate(rf_grid_cv.best_estimator_, X_train, y_train, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

In [ ]:
#oh wait, but so now, this time, as opposed to last time, WE'RE NOT SPLITTING INTO TRAINING & TEST GROUPS!!!
#cuz we've already done that! and that's why we have that best rf model stored!
#so now, we're, satisfied w/ our model/best one yet/so far, and so thus applying it to / expanding it to train on /
#training it on the ENTIRE dataset MINUS/EXCEPT OUR TARGET - BIG MOUNTAIN!

In [ ]:
#the function we made above includes a 'test_scores' column using the type of scoring we specified/requested, which was
#neg MAE, and so that's what these are!
cv_results['test_score']

In [ ]:
#very close to template

In [ ]:
mae_mean, mae_std = np.mean(-1 * cv_results['test_score']), np.std(-1 * cv_results['test_score'])
mae_mean, mae_std

In [ ]:
#again, super duper close

These numbers will inevitably be different to those in the previous step that used a different training data set. They should, however, be consistent. It's important to appreciate that estimates of model performance are subject to the noise and uncertainty of data!

In [ ]:
#oh yeah! so the reason they're different from unit 4's is because this uses the ENTIRE dataset (minus Big M) as the
#training set whereas that only used 70% (and was also minus BM)

## 5.7 Calculate Expected Big Mountain Ticket Price From The Model<a id='5.7_Calculate_Expected_Big_Mountain_Ticket_Price_From_The_Model'></a>

In [ ]:
#okay so now we're homing in on JUST BM
#pinpoint just its row and taking as the X predictors all its columns/features except for of course price - the thing
#we're tryna predict!

#and then of course the y is that price iA we'll predict
X_bm = ski_data.loc[ski_data.Name == "Big Mountain Resort", model.X_columns]
y_bm = ski_data.loc[ski_data.Name == "Big Mountain Resort", 'AdultWeekend']

In [ ]:
X_bm

In [ ]:
y_bm

In [ ]:
y_bm.values

In [ ]:
y_bm.values.item()

In [ ]:
#ahh okay, so the above explains what is going on here / what these things mean. see below

In [ ]:
model.predict(X_bm)

In [ ]:
#so we're storing as a variable / telling it to use our best rf model to predict the price using the X_bm we just defined
#note, as learned above, using '.item()' is SIMPLY to nicely isolate the single number that's trapped/wrapped in an
#array brackets and extra/neous info and cherry-pick out just the single number itself! for clean easy viewing
bm_pred = model.predict(X_bm).item()

#but how does it know what it's predicting? like how does it know to use all those like 32 params and use it to come
#up w/ the target price number? like doesn't it have to know the relationships between the training set's same set of
#these 32 variables and then their prices?


#ohhh yeah, so that's already all baked deep into the model, if you look back / trace back its roots/history!

#that's why it was so good to do that intense, packed building package work earlier, cuz now it's as simple as just
#calling on one simple word/variable, representing all of it - MODEL!


In [ ]:
bm_pred

In [ ]:
#there we go! that's the predicted price! i.e., based on what everyone else is doing given their offerings, this is
#what Big Mountain should charge, i.e. the FAIR MARKET PRICE

#by contrast, this is what they were charging:

In [ ]:
#as seen above, all this is simply doing is isolating down to the number itself; don't get confused / overwhelmed;
#nothing fancy is happening here

y_bm = y_bm.values.item()
y_bm

In [ ]:
# wow! so subhanaAllah they were charging $15 too less!/UNDERcharging / selling themselves short by $15 per person!!!
#that's almost 20% short/under!!!!

In [ ]:
print(f'Big Mountain Resort modelled price is ${bm_pred:.2f}, actual price is ${y_bm:.2f}.')
print(f'Even with the expected mean absolute error of ${mae_mean:.2f}, this suggests there is room for an increase.')

In [ ]:
#our number here is slightly more than template, but again, it's gonna be slightly different w/ each run. we could
#run it like 5 times and take the average, right?

#cross_validate(model,X, cv=5)

#....^No.... at least not like this. not sure how you'd do it


######################################################################
##########################???QUESTION???##############################
######################################################################

In [ ]:
# and oh right, good point - to look at the MAE, which is about $10. so that's saying even if we're wrong and max out
# that mean error, we're STILL a few bucks above where they're at!!

This result should be looked at optimistically and doubtfully! The validity of our model lies in the assumption that other resorts accurately set their prices according to what the market (the ticket-buying public) supports. The fact that our resort seems to be charging that much less than what's predicted suggests our resort might be undercharging. 
But if ours is mispricing itself, are others? It's reasonable to expect that some resorts will be "overpriced" and some "underpriced." Or if resorts are pretty good at pricing strategies, it could be that our model is simply lacking some key data? Certainly we know nothing about operating costs, for example, and they would surely help.

In [ ]:
# right, so this assumes that OTHER resorts are basing their prices on features and market norms. but if everyone did that,
# #how could ANYONE set their prices, like who would go first, and HOW would they go first?? #headscratcher

#oh yeah, good point - maybe other resorts are mispricing themselves too cuz they don't know / maybe didn't base it off
#of what others were charging! or maybe only based it off of other SIMILAR groups. BASICALLY LOOKING AT ***COMPS***
# / NEAREST NEIGHBORS / ***CLUSTERS*** thru PCA!!!! that's maybe how these algorithms work!!! how many bedrooms, bathrooms, *newness* /
# wildcard, garage, yard size, neighborhood, etc etc. but like w/ those too, maybe they weren't basing it off of what
#others were going for!


#oh yeah, and that's a key point - about operating costs. of course this is a huge factor in any business's pricing
#strategy and we know NOTHING about this for ANY of our 

## 5.8 Big Mountain Resort In Market Context<a id='5.8_Big_Mountain_Resort_In_Market_Context'></a>

Features that came up as important in the modeling (not just our final, random forest model) included:
* vertical_drop
* Snow Making_ac
* total_chairs
* fastQuads
* Runs
* LongestRun_mi
* trams
* SkiableTerrain_ac

A handy glossary of skiing terms can be found on the [ski.com](https://www.ski.com/ski-glossary) site. Some potentially relevant contextual information is that vertical drop, although nominally the height difference from the summit to the base, is generally taken from the highest [_lift-served_](http://verticalfeet.com/) point.

In [ ]:
# okay so i think it's saying vertical drop is the distance b/w the highest and lowest 'lift-served' points,
# and 'lift-served' simply means the points/verticalities at which the lift drops off at?

It's often useful to define custom functions for visualizing data in meaningful ways. The function below takes a feature name as an input and plots a histogram of the values of that feature. It then marks where Big Mountain sits in the distribution by marking Big Mountain's value with a vertical line using `matplotlib`'s [axvline](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.axvline.html) function. It also performs a little cleaning up of missing values and adds descriptive labels and a title.

In [ ]:
# hey they copied me! this is the same idea of what i did earlier on w/ my plots and tables!

In [ ]:
#Code task 1#
#Add code to the `plot_compare` function that displays a vertical, dashed line
#on the histogram to indicate Big Mountain's position in the distribution

#Hint: plt.axvline() plots a vertical line, its position for 'feature1'

#would be `big_mountain['feature1'].values, we'd like a red line, which can be

#specified with c='r', a dashed linestyle is produced by ls='--', (#'ls' = 'line style!')

#and it's nice to give it a slightly reduced alpha value, such as 0.8.

#Don't forget to give it a useful label (e.g. 'Big Mountain') so it's listed in the legend


def plot_compare(feat_name, description, state=None, figsize=(10, 5)):
#so remember, a function is like a food making kit, and it bundles all the ingredients together in the beginning,
#to lay it out for the recipe. so you're gonna hand it:  (explained below) the feature/column name, a description
#of that feature, the specific state (like Pennsylvania), and the fig_size that'll be used for plotting


#so this is gonna result in a histo of the feature for ALL resorts, including BM, and we're gonna mark it to see where
#it lies
    
    """Graphically compare distributions of features.
    
    Plot histogram of values for all resorts and reference line to mark
    Big Mountain's position.
    
    Arguments:
    feat_name - the feature column name in the data
    description - text description of the feature
    state - select a specific state (None for all states)
    figsize - (optional) figure size
    """
    
    plt.subplots(figsize=figsize)
    
    #okay so apparently there's this quirk w/ hist, described below, where it sometimes doesn't like NaNs
    # quirk that hist sometimes objects to NaNs, sometimes doesn't
    # filtering only for finite values tidies this up
    
    if state is None:
        ski_x = ski_data[feat_name]  #so if you DON'T specify a single state, this will just filter the ski_data down 
    else:                            #to just the column of the one specified feature and include ALL resorts/states
                                     #since you didn't filter so it won't subset any further
        ski_x = ski_data.loc[ski_data.state == state, feat_name]  #by contrast, if you DO specify a state, it'll take
    ski_x = ski_x[np.isfinite(ski_x)]                           #it a step further and subset not just for/to that column,
                                                                #but for that specific state!
    plt.hist(ski_x, bins=30)
    plt.axvline(x=big_mountain[feat_name].values, c='r', ls='--', alpha=0.8, label='Big Mountain')
    #so obvy this only applies if you're doing ALL resorts/states/not specifying or are doing JUST the state of Montana!
    
    #so we have to tell it which x do dash-the-dotted line vertical drawn on - so the x- point @which/=equal to Big Mountain's value of that specific/particular feature!
    #cool so we're not literally telling it to find the point/plot of Big Mountain and mark/highlight that, we got
    #creative, did a natural workaround - we said take the VALUE OF BIG MOUNTAIN AND SIMPLY MARK THAT VALUE BAR!!!
    plt.xlabel(description)
    plt.ylabel('frequency')
    plt.title(description + ' distribution for resorts in market share')
    plt.legend()

### 5.8.1 Ticket price<a id='5.8.1_Ticket_price'></a>

Look at where Big Mountain sits overall amongst all resorts for price and for just other resorts in Montana.

In [ ]:
#okay sick. slick. so now we're gonna put this function to use and see where BM stands w/ price:

plot_compare('AdultWeekend', 'Adult weekend ticket price ($)')

In [ ]:
# okay cool so we can see it's a bit towards the higher end, althought there's def several one-off outliers that push
# way even further upstream

In [ ]:
#now let's see how JUST Montana looks like and how Big Mountain stacks there
#btw MINE WAS WAY BETTER - I actually colored the BAR and was SPECIFIC TO BIG MOUNTAIN! plotted on that basis,
#i.e. one plot for EACH resort, NOT a bucketized histo!
#in this case, when just looking at one state, they bucketized in groups of 1's or 2's
plot_compare('AdultWeekend', 'Adult weekend ticket price ($) - Montana only', state='Montana')

In [ ]:
#ouch. so yeah, we saw this earlier - BM is by far the most expensive in Montana - AND WE'RE TRYNA TELL THEM TO CHARGE
#EVEN MORE! and remem, when we compared its features to other ones in Montana it offered WAY LESS!

### 5.8.2 Vertical drop<a id='5.8.2_Vertical_drop'></a>

In [ ]:
plot_compare('vertical_drop', 'Vertical drop (feet)')

In [ ]:
#ah okay, so yeah you can go about these plots this way too, of doing histogram and bucketizing and doing frequency vs.
#value. but i like doing individual plots of each resort and make y-axis the actual value and order it descending

#but yeah, okay, vert drop is towards the higher side so it's putting its money where its mouth is

Big Mountain is doing well for vertical drop, but there are still quite a few resorts with a greater drop.

### 5.8.3 Snow making area<a id='5.8.3_Snow_making_area'></a>

In [ ]:
plot_compare('Snow Making_ac', 'Area covered by snow makers (acres)')

In [ ]:
# good job BM! top performer!

Big Mountain is very high up the league table of snow making area.

### 5.8.4 Total number of chairs<a id='5.8.4_Total_number_of_chairs'></a>

In [ ]:
plot_compare('total_chairs', 'Total number of chairs')

In [ ]:
# def on the high end of the normal range. beyond this are freak outliers

In [ ]:
#so now we're starting to get an idea of why it scored so high / why it was told to charge such a premium--
#cuz it did so well in the most important categories!!
#that's what we're doing here! we saw what categories were important and we made a function to quickly see how well
#BM did in all of them!
#I'm sure we could set it up so that we can just make a list of these like 8 features and define a plot grid of like
#4x2 and pass the list to it to make one plot for each feature

#also though, since Montana is a tough league/division, we should see how the REST OF MONTANA did w/ price predictions
#based on their performance in important categories / weight scores / score weights!

Big Mountain has amongst the highest number of total chairs, resorts with more appear to be outliers.

### 5.8.5 Fast quads<a id='5.8.5_Fast_quads'></a>

In [ ]:
plot_compare('fastQuads', 'Number of fast quads')

In [ ]:
# okay that's good, def ahead of most

# and remem, the weights are in order as laid out earlier, which we found in unit 4 w/ the COEFS!
# remem, these coeffs are according to the LINEAR REGRESSION model! the random forest model ranking IMPORTANCE / 
# best estimators are basically the same features but in a different order/weight proportions (on the right side)
# and note the one we're going w/, for some reason, based on the list/order they provided above and reproduced here
# on the left below, is based on LINEAR REGRESSION, although i thought RANDOM FOREST/rf was better and that's what we
# went w/ in the end as our 'best model'. but i guess since the top features, regardless of order, were essentially the
# same, it doesn't matter so much, the point here is to simply explore the histogram/distribution / where-BM-stands
# in the ranks amongst the population

# LINEAR REGRESSION/lr                   # RANDOM FOREST/rf (NOTE: All feature importances together add up to 1!!)
# vertical_drop        10.767857         # fast_Quads .26
# Snow Making_ac        6.290074         # Runs       .25
# total_chairs          5.794156         # Snow Making.11
# fastQuads             5.745626         # vert drop  .09
# Runs                  5.370555         # skiable ter.03
# LongestRun_mi         0.181814         # tot chairs .02
# trams                -4.142024         # days open. .02
# SkiableTerrain_ac    -5.249780         # daysopstrat.02

#okay, i guess i could see why the lr ones are better to go w/, cuz of end
#but at same time, what's up w/ the negatives that early on??

Most resorts have no fast quads. Big Mountain has 3, which puts it high up that league table. There are some values  much higher, but they are rare.

In [ ]:
#oh yeah wow just realized most don't have any! that's what that big bar is - it's not 1!

### 5.8.6 Runs<a id='5.8.6_Runs'></a>

In [ ]:
plot_compare('Runs', 'Total number of runs')

In [ ]:
#nicely done BM, nicely done. top-tier once again


#btw, would be nice if it said / marked the actual number that it was highlighting for BM value
#cuz can't always exactly tell

Big Mountain compares well for the number of runs. There are some resorts with more, but not many.

### 5.8.7 Longest run<a id='5.8.7_Longest_run'></a>

In [ ]:
plot_compare('LongestRun_mi', 'Longest run length (miles)')

In [ ]:
#wow, again! it seems to have a similar place in all of these, at the top of the normal range, just before the mind freaks /
#at the very bottom/start of the insane freaks

Big Mountain has one of the longest runs. Although it is just over half the length of the longest, the longer ones are rare.

### 5.8.8 Trams<a id='5.8.8_Trams'></a>

In [ ]:
plot_compare('trams', 'Number of trams')

In [ ]:
# no prob! you're def not alone! that's the overwhelming norm!

The vast majority of resorts, such as Big Mountain, have no trams.

### 5.8.9 Skiable terrain area<a id='5.8.9_Skiable_terrain_area'></a>

In [ ]:
plot_compare('SkiableTerrain_ac', 'Skiable terrain area (acres)')

In [ ]:
# FIXED NOW & MATCHES -- totally forgot about the Silverton Mountain Colorado correction!! made a huge diff!! that one mistake alone was what threw off my r2 score completely in part 3 and sent me on a / led me down a wild goose chase!!! unresolved until now / until this tipped me off!! instantly shrunk / transformed the preload!

# >>! ski_data.loc[ski_data.Name=='Silverton Mountain', 'SkiableTerrain_ac'] = 1819 !

#-- but prior:

#interesting - so mine includes the one freak w/ the crazy amount of skiable terrain in Colorado, one that we investigated
#in the very beginning and decided to keep. but the template doesn't have it...

In [ ]:
# sveeet!! right now

In [ ]:
ski_data.copy()[['state','SkiableTerrain_ac']].sort_values(by='SkiableTerrain_ac', ascending=False).head()

In [ ]:
# yep, confirmed, they took out that one resort. but they def decided to keep it in and keep it that huge high value
# in the beginning, soooerrrr...

# ohhhh okay, so went back and looked. so, yes, we DID decide to keep that resort, **BUT**, we DID change it to a
# much lower value!!! upon research

In [ ]:
# yeah so BM is wayyy, way up there!! in the big sky! near the top of the big mountain of graph!

Big Mountain is amongst the resorts with the largest amount of skiable terrain.

## 5.9 Modeling scenarios<a id='5.9_Modeling_scenarios'></a>

Big Mountain Resort has been reviewing potential scenarios for either cutting costs or increasing revenue (from ticket prices). Ticket price is not determined by any set of parameters; the resort is free to set whatever price it likes. However, the resort operates within a market where people pay more for certain facilities, and less for others. Being able to sense how facilities support a given ticket price is valuable business intelligence. This is where the utility of our model comes in.

The business has shortlisted some options:
1. Permanently closing down up to 10 of the least used runs. This doesn't impact any other resort statistics.
2. Increase the vertical drop by adding a run to a point 150 feet lower down but requiring the installation of an additional chair lift to bring skiers back up, without additional snow making coverage
3. Same as number 2, but adding 2 acres of snow making cover
4. Increase the longest run by 0.2 mile to boast 3.5 miles length, requiring an additional snow making coverage of 4 acres

The expected number of visitors over the season is 350,000 and, on average, visitors ski for five days. Assume the provided data includes the additional lift that Big Mountain recently installed.

In [ ]:
# okay and it has about 100 runs
# okay so they'd increase the vert by increasing the LOW POINT / DEPTH / going DOWN, but then this'd be ADDING a run
# and also they'd need to add a lift to get ppl back up to ground/sea level. and this would not add to the snow making coverage?
# okay #3 is the same option as above but WOULD add partial / make some of it covered w/ snow
# increasing the longest run by .2 miles, but that doesn't seem like that would really make it jump by any leaps or bounds
# by any means. i mean it's at an interesting position where it's already near the top, but not in the most elite of
# elite categories, so there's this gap / deadspace b/w/ it's tier and the superperformers. Getting to 3.5 would put it
# at the top of its current class, but it still wouldn't cross the bridge/bridge the gap into the next territory
# so it's a toss-up. and that added snow making acreage that would come of it wouldn't really make a dent in the overall
# distribution picture

In [ ]:
expected_visitors = 350_000

In [ ]:
#oh wow that's a lot of visitors. really? they stay 5 days? okay i guess makes sense, cuz they're going all that way
#out to the middle of nowhere. and okay, we're saying that the data we have includes that they already
#installed that extra lift

In [ ]:
#okay cool so this is grouping all 8 key / most important features in 1
all_feats = ['vertical_drop', 'Snow Making_ac', 'total_chairs', 'fastQuads', 
             'Runs', 'LongestRun_mi', 'trams', 'SkiableTerrain_ac']
big_mountain[all_feats]

#Thus this^ is gonna give us just these 8 key features of big mountain
#nice view:

In [ ]:
#Code task 2#
#In this function, copy the Big Mountain data into a new data frame
#(Note we use .copy()!)   #>>Yes!! gotta make sure we do that! otherwise it's gonna yell at you

#And then for each feature, and each of its deltas (changes from the original),

#create the modified scenario dataframe (bm2) and make a ticket price prediction

#for it. The difference between the scenario's prediction and the current

#prediction is then calculated and returned.

#Complete the code to increment each feature by the associated delta

def predict_increase(features, deltas):
    
    #ohh okay, so WE'RE specifying the deltas, i.e. manually changing the values 
    """Increase in modelled ticket price by applying delta to feature.
    
    Arguments:
    features - list, names of the features in the ski_data dataframe to change
    deltas - list, the amounts by which to increase the values of the features
    
    Outputs:
    Amount of increase in the predicted ticket price
    """
    
    bm2 = X_bm.copy()   #okay this creates a copy of X_bm, which are the predicting features of BM, so doesn't include price
    
    #okay, so this is taking two things - features & deltas. so each item in features is 'f' and each item in
    #deltas is 'd'
    #so yeah, those are the two things you're feeding it from up there in the function call
    
    #so remember, zip() takes 2 lists of equal length and makes them into tuple pairs, haha it's like a 'ZIPPER' that
    #takes the two independent equal-length lines and 'zips' them together into one unified! it's like each nook is
    #paired w/ another on the other side buddy-buddy!
    
    for f, d in zip(features, deltas): #oh i see so we use zip so that it can go thru two lists at once and / but
        bm2[f] += d                   #we can reference them independently w/ two diff variables
    return model.predict(bm2).item() - model.predict(X_bm).item()



# okay so this is going thru the set of features in features, one by one, 'f by f', and adding the delta to it,
# each delta d in the list delta, which we provide

#so this finishes going through each f,d pair, adjusting each f feature by each corresponding d delta

#so the output is gona be the NEW PREDICTED PRICE based on these new / delta'd variables!!

#oh actually, wait - CAREFUL - it's not giving you the price ITSELF - it would if it stopped at / was just:
# return model.predict(bm2).item()

#but it's subtracting the ORIGINAL predicted price from it

#so that means what it's giving you is the PRICE PREDICTION DELTA!!!!!

#i.e. what this is showing you is what delta do you get in price for your delta(s) in the other variables????

#cuz remem the whole point is to see how different the price looks based on changes in variables,
#so we wanna see what the model is most sensitive to
#it should be in line w/ the RANDOM FOREST chart we made above/earlier


### 5.9.1 Scenario 1<a id='5.9.1_Scenario_1'></a>

Close up to 10 of the least used runs. The number of runs is the only parameter varying.

In [ ]:
#okay so in this scenario, the ONLY feature we're changing is the number of runs, so that's gonna go down by -10

In [ ]:
[i for i in range(-1, -11, -1)]

In [ ]:
runs_delta = [i for i in range(-1, -11, -1)]  #so we're diminishing the runs by 1 down at a time till we're 10 down from original
price_deltas = [predict_increase(['Runs'], [delta]) for delta in runs_delta]


#okay so remem 'predict_increase' is our custom/homebrew function/homemade/handmade
#where we take (features, deltas) as the arguments
#so here they're giving us ['Runs'] as the only feature
#and [delta], which refers to each item at a time in the range function from -1 to -10
#so remem it's bm2[f] += d, so it's gonna 'add -1:-10' aka subtract, so we're good/straight there

#so since the features arg is just ONE feature with ONE value, it's just gonna stay constant that value, it's as if
#it converts that to a list of itself / duplicates it/copies/drags down

#and so since it's iterating thru the range, and we're only interested in the last value, we're only gonna focus on /
#care about the last value in the list!


#but, in response to that - my question is - wasn't it a lot simpler? like couldn't we have straight just made delta = -10?????
#just like how we're holding feature constant @/for Runs????

#i guess one thing is this let's us keep it a consistent way for each scenario, i.e. use the same/one function
#and the second thing is it let's us see the gradual change / trend over 'time' / w/ each addl unit change

#we'll test it out next

#and remember, these are price DELTAS!!!! NOT the new predicted prices themselves!! cuz we wanna clearly / right away
#see immediately what's happening / how the original predicted price is being affected / what's its relationship to
#changes in this/these other variables -- is it going down (like when they're going down)

#remember that the delta formula is new_prediction - old_/original_prediction,
#so if that delta is POSITIVE+, that means the predicted price is INCREASING in response!
#and if the delta is NEGATIVE-, that means the predicted price is DECREASING/going down in response!

In [ ]:
#so here's how the price predictions are being affected by changes in the #of runs / how the new price predictions
#are/change in response to changes/(decreases) in runs

price_deltas

In [ ]:
predict_increase(['Runs'], [-10])

In [ ]:
#DICE!!!!! you were right! you did it!!!

In [ ]:
# okay so there you have it, it looks like Runs is a very insensitive field! even after dropping the full ***10*** runs
# the predicted price only went down $1.81!!!

# it seems to work by/on step-changes. notice the 1st number in the set is Zero/0/jeero - that's for ONE closed run!
# aka closing down ONE run shouldn't have ANY effect on the price!! and similarly you ca see other steps in the set

In [ ]:
#Code task 3#
#Create two plots, side by side, for the predicted ticket price change (delta) for each
#condition (number of runs closed) in the scenario and the associated predicted revenue
#change on the assumption that each of the expected visitors buys 5 tickets   #>>cuz 5 day stays each!!!
#There are two things to do here:
#1 - use a list comprehension to create a list of the number of runs closed from `runs_delta`
#2 - use a list comprehension to create a list of predicted revenue changes from `price_deltas`
runs_closed = [-1 * d for d in runs_delta] #1 #this is simply negating the / each number in runs_delta cuz now we're talking about the NUMBER of runs closed, so need a positive number, not a mathematical operator
fig, ax = plt.subplots(1, 2, figsize=(10, 5)) # >> this is that side-by-side plot they want, a '1x2' configuration!!!
fig.subplots_adjust(wspace=0.5)

#okay so this is for the first plot (ax[0] out of [0] & [1])
#we're plotting: x: runs_closed - 1:10  vs.  y: price_deltas
#so we're gonna show how price is changed/affected by changes in runs (decreases)
#price deltas we're simply borrowing/reusing/recycling directly from the original, as-is

ax[0].plot(runs_closed, price_deltas, 'o-')
ax[0].set(xlabel='Runs closed', ylabel='Change ($)', title='Ticket price')



#okay and for the second plot, we're tryna show the predicted REVENUE CHANGE!
#so we're assuming that the number of visitors holds constant throughout all these @ 350,000

#okay, so what this is gonna give us is the amount of revenue per each lift loss amt!

#so 5 days/visitor * 350,000 visitors * price delta
#so this is gonna tell us for each run closed, how much the total REVENUE changes!
#we know how much the TICKET PRICE changes, so the total revenue for the season is just the total number of visitors
#per the season * the number of days each visitor will stay there on average * price per ticket per/each day!
#and so likewise the revenue DELTA will be the total number of visitors per the season * the number of days
#each visitor will stay there on average * PRICE DELTA per ticket per/each day!
#and so we have these price deltas @ each add'l closed run

#okay so it's gonna iterate/list comprehension thru / for each ticket price delta, which is based on, in order, each
#closed run, so thus, it's already in order, and so when we do the progressive plot of revenue delta over the number
#of runs closed, it's already tied to / in the right order to the runs closed -> the corresponding price delta!!!


revenue_deltas = [5 * expected_visitors * p for p in price_deltas] #2

ax[1].plot(runs_closed, revenue_deltas, 'o-')
ax[1].set(xlabel='Runs closed', ylabel='Change ($)', title='Revenue');

In [ ]:
#btw, to give this context, in absolute numbers, since this can be misleading since it's ONLY talking about deltas:
#@ 350,000 visitors, for 5 days each, @ $96/day, that's $168 MILLION IN REVENUE! so you might've saw this and thought,
#oh man, i thought we were only losing $1.81 a ticket, but now we're losing $3M+!!!

#but think about it. that's what we expected/knew - that comes from: $1.81 * 350,000 * 5 = $3.1M+ loss. BUT - now we
#know how much we're actually making! $168M!!! so even if we lose $3M / $168M, that's still only 1.8%!!!!
#and will prob have a bunch of savings in / from other ways!!!

#and notice that it does make sense that the two graphs will be the exact same shape cuz they're based on the same thing
#the left is simply at the individual scale and the right is the total
#so the right is simply the left multiplied by a CONSTANT COEFFICIENT (5 * 350,000), (i.e. the number of tickets per
#season!!!) / THE SAME COEFFICIENT AT EACH POINT!!!

#like we showed above: @Step 1, across both:  Left: -$1.81 <> Right: $1.81 * 5 * 350,000 = -$3.1M!


The model says closing one run makes no difference. Closing 2 and 3 successively reduces support for ticket price and so revenue. If Big Mountain closes down 3 runs, it seems they may as well close down 4 or 5 as there's no further loss in ticket price. Increasing the closures down to 6 or more leads to a large drop. 

In [ ]:
# interesting point - if gonna close 3, might as well close 5. would have to weigh diff things. would #runs affect
# capacity? if so, would it be better to keep capacity smaller so you can charge more per ticket (throttle supply)
# or would more capacity be more advantageous cuz then could bring in more people and sell *MORE* tickets?
# but they did say specifically that changing the number of runs WOULD NOT affect any other parameter, like skiable
# acres etc, so i guess we'll go w/ that

### 5.9.2 Scenario 2<a id='5.9.2_Scenario_2'></a>

In this scenario, Big Mountain is adding a run, increasing the vertical drop by 150 feet, and installing an additional chair lift.

In [ ]:
#okay so this is the one about increasing the vert drop by 150 feet by going / expanding that much LOWER, and this
#requires adding another run to do that AS WELL AS another lift - to bring em back up!!

In [ ]:
#Code task 4#
#Call `predict_increase` with a list of the features 'Runs', 'vertical_drop', and 'total_chairs'
#and associated deltas of 1, 150, and 1

#oh i just realized we call it "predict increase" but it could be that the predicted price is going DOWN! which is
#exactly what happened in the last scenario!! but really it's the price CHANGE aka price DELTA/delter!


#predict the price increase (CHANGE 'TRIANGLE' DELTA!) based on the following changes to / in the following features
ticket2_increase = predict_increase(['Runs', 'vertical_drop', 'total_chairs'], [1, 150, 1])
#okay cool so it's a change of +1 run, +150 ft vertical drop, and +1 chair!!


revenue2_increase = 5 * expected_visitors * ticket2_increase

# so the revenue -- again, *CHANGE/DELTA* -- for the season is gonna be 5 * 350,000 expected visitors * the change 
# per each ticket, i.e. the thing we just calculated right before that --> ticket2_increase!

In [ ]:
print(f'This scenario increases support for ticket price by ${ticket2_increase:.2f}')
print(f'Over the season, this could be expected to amount to ${revenue2_increase:.0f}')

### 5.9.3 Scenario 3<a id='5.9.3_Scenario_3'></a>

In this scenario, you are repeating the previous one but adding 2 acres of snow making.

In [ ]:
#Code task 5#
#Repeat scenario 2 conditions, but add an increase of 2 to `Snow Making_ac`
ticket3_increase = predict_increase(['Runs', 'vertical_drop', 'total_chairs', ___], [1, 150, 1, ___])
revenue3_increase = 5 * expected_visitors * ticket3_increase

In [ ]:
print(f'This scenario increases support for ticket price by ${ticket3_increase:.2f}')
print(f'Over the season, this could be expected to amount to ${revenue3_increase:.0f}')

Such a small increase in the snow making area makes no difference!

### 5.9.4 Scenario 4<a id='5.9.4_Scenario_4'></a>

This scenario calls for increasing the longest run by .2 miles and guaranteeing its snow coverage by adding 4 acres of snow making capability.

In [ ]:
#Code task 6#
#Predict the increase from adding 0.2 miles to `LongestRun_mi` and 4 to `Snow Making_ac`
predict_increase([___, ___], [___, ___])

No difference whatsoever. Although the longest run feature was used in the linear model, the random forest model (the one we chose because of its better performance) only has longest run way down in the feature importance list. 

## 5.10 Summary<a id='5.10_Summary'></a>

**Q: 1** Write a summary of the results of modeling these scenarios. Start by starting the current position; how much does Big Mountain currently charge? What does your modelling suggest for a ticket price that could be supported in the marketplace by Big Mountain's facilities? How would you approach suggesting such a change to the business leadership? Discuss the additional operating cost of the new chair lift per ticket (on the basis of each visitor on average buying 5 day tickets) in the context of raising prices to cover this. For future improvements, state which, if any, of the modeled scenarios you'd recommend for further consideration. Suggest how the business might test, and progress, with any run closures.

**A: 1** Your answer here

## 5.11 Further work<a id='5.11_Further_work'></a>

**Q: 2** What next? Highlight any deficiencies in the data that hampered or limited this work. The only price data in our dataset were ticket prices. You were provided with information about the additional operating cost of the new chair lift, but what other cost information would be useful? Big Mountain was already fairly high on some of the league charts of facilities offered, but why was its modeled price so much higher than its current price? Would this mismatch come as a surprise to the business executives? How would you find out? Assuming the business leaders felt this model was useful, how would the business make use of it? Would you expect them to come to you every time they wanted to test a new combination of parameters in a scenario? We hope you would have better things to do, so how might this model be made available for business analysts to use and explore?

**A: 2** Your answer here